# Word2Vec

In [1]:
import re 
import pandas as pd

from nltk.corpus import stopwords
stopwords_sp = stopwords.words('spanish')

def pre_procesado(texto):
    texto = texto.lower()
    texto = re.sub(r"[\W\d_]+", " ", texto)
    texto = " ".join([palabra for palabra in texto.split() if palabra not in stopwords_sp])
    return texto.split()


In [7]:
rap = pd.read_csv("rap_espanol.csv")
rap['pp'] = rap['letra'].apply(lambda texto: pre_procesado(texto))
rap.head()

,id,artista,cancion,album,letra,anyo,visitas,pp
0,0,Denom,Machete (con Jarfaiter y Gente jodida),Medicina,"Para su nuevo disco ""Medicina"", Denom ha vuelt...",2019,126,"[nuevo, disco, medicina, denom, vuelto, contar..."
1,1,Denom,Vacío (con Ivo Incuerdo),Medicina,"[Denom]\nYo que quería, yo que pedía vida,\nSe...",2019,361,"[denom, quería, pedía, vida, partió, mitad, ah..."
2,2,Denom,El orgullo es fiel (con Juancho Marqués y Elio...,Medicina,"""El orgullo es fiel"" es uno de los cortes incl...",2019,262,"[orgullo, fiel, cortes, incluidos, nuevo, disc..."
3,3,Denom,Mueve mueve (con Fernandocosta),Medicina,"[Estribillo: Denom] (x2)\nMueve, mueve, mueve,...",2019,578,"[estribillo, denom, x, mueve, mueve, mueve, ti..."
4,4,Jaro Desperdizio,Insomnia,"Sin álbum, es un vídeo suelto","[Estribillo]\nY en esta noche, ¿Quién me arrop...",2019,219,"[estribillo, noche, quién, arropará, si, invie..."


In [ ]:
#!pip install gensim

In [18]:
%%time
import gensim.models.word2vec as w2v

mi_modelo = w2v.Word2Vec(rap['pp'].values,
                        sg=1, # 1 para usar skip-gram, 0 para usar CBOW
                        seed=1, # 1 para tener resultados reproducibles y debuggear
                        vector_size=256, # el tamaño de los vectores de palabras o neuronas
                        min_count=50, # mínimo de veces que ha de aparecer una palabra para ser considerada
                        window=12, # ventana contextual de cada palabra
                        )

Wall time: 2min 51s


In [19]:
mi_modelo.wv.most_similar("familia")

[('cla', 0.5168694257736206),
 ('biblia', 0.5087817311286926),
 ('padre', 0.39692389965057373),
 ('madre', 0.3946682810783386),
 ('amigos', 0.3883940875530243),
 ('familiares', 0.384700208902359),
 ('callejón', 0.37463513016700745),
 ('padres', 0.37300822138786316),
 ('hermanos', 0.36924976110458374),
 ('amistad', 0.36864331364631653)]

In [24]:
def similitud(inicio1, fin1, inicio2):
    fin2 = mi_modelo.wv.most_similar_cosmul(
    positive=[inicio2, inicio1],
    negative=[fin1])[0][0]
    return f"{inicio1} es a {fin1} como {inicio2} es a {fin2}"

similitud("amar", "amor", "odio")

'amar es a amor como odio es a odiar'

In [57]:
# len(mi_modelo.wv)
# mi_modelo.wv.key_to_index
# mi_modelo.wv.index_to_key

# matrix = pd.DataFrame(mi_modelo.wv.key_to_index.items())
# matrix.index = mi_modelo.wv.key_to_index.keys()
# matrix

TypeError: 'Word2Vec' object is not subscriptable

In [ ]:
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt

# pip install plotly
import plotly.graph_objs as go
from plotly.offline import iplot, init_notebook_mode

%matplotlib inline
init_notebook_mode(connected=True)



X = mi_modelo[mi_modelo.wv.vocab] # mi_modelo[mi_modelo.wv.key_to_index]
matrix = pd.DataFrame(X)

matrix.index = mi_modelo.wv.vocab.keys() # mi_modelo.wv.key_to_index.keys()
matrix


pca = PCA(n_components=2)

result = pca.fit_transform(matrix)
result = pd.DataFrame(result)
result.columns = ['X', 'Y']
result['Palabra'] = matrix.index.values


trace = go.Scatter(x=result['X'].values,
                   y=result['Y'].values,
                   text=result['Palabra'].values,
                   mode='markers',
                   name="vocabulario") 

layout = go.Layout(title="PCA")

fig = go.Figure(data=[trace], layout=layout)
iplot(fig)